# Download PointData for Magnaprobe data within bounding box 

### Get database session 

In [1]:
# Import the function to get connect to the db
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData
from snowexsql.conversions import query_to_geopandas, query_to_pandas
from snowexsql.db import get_table_attributes

# Import some other tools 
# from datetime import date
import datetime
from sqlalchemy import inspect
import shapely.geometry
from geoalchemy2.shape import from_shape
import geoalchemy2.functions as gfunc

# load the database
db_name = 'snow:hackweek@db.snowexdata.org/snowex'
# engine, session = get_db(db_name)

# print('SnowEx Database successfully loaded!')

### Query the DB to see what tables are available

In [2]:
# Using the function get_db, we receive 2 ways to interact with the database
engine, session = get_db(db_name)

# Output the list of tables in the database 
tables = inspect(engine).get_table_names()
print(tables)

session.close()

['spatial_ref_sys', 'points', 'layers', 'sites', 'images']


### Query a table to see what columns to use 

In [3]:
# # Import the function to investigate a table
# from snowexsql.db import get_table_attributes 

# # Use the function to see what columns are available to use. 
# db_columns = get_table_attributes(LayerData) 
# print("These are the available columns in the table:\n \n* {}\n".format('\n* '.join(db_columns)))


### Add Bounding Box 

In [4]:
# How to pull out ALL point data from the database that falls within our box
bbox_WSEN = 742000, 4322000, 747000, 4325000 # EPSG 26912?
x1, y1, x2, y2 = bbox_WSEN
polygon = shapely.geometry.Polygon([[x1, y1], [x1, y2], [x2, y2], [x2, y1]]) # used box() before
wkb_element = from_shape(polygon, srid=26912) # which srid is right?


### Filter Data

In [5]:
# Instrument name 
instrument = "magnaprobe" 

# Get a session
engine, session = get_db(db_name)

# The part inside the query function is what we want back, in this case all columns for the point data
qry = session.query(PointData.geom, PointData.easting, PointData.northing, PointData.date, PointData.value, 
                    PointData.type, PointData.instrument)

# Filter by bounding box 
qry = qry.filter(gfunc.ST_Within(PointData.geom, wkb_element))

# Filter by an instrument 
qry = qry.filter(PointData.instrument == instrument) # .in_(['magnaprobe', 'pit_ruler']))

# Slicing the dataset for specified dates 
date1 = datetime.date(2020,1,29)
date2 = datetime.date(2020,2,6)
qry = qry.filter(PointData.date.in_([date1, date2]))

# Execute the query and convert to geopandas in one handy function
df = query_to_geopandas(qry, engine) #directly pass to geopandas dataframe
print(df.head())

# how many did we retrieve?
print(f'{len(df.index)} records returned!')



                             geom        easting      northing        date  \
0  POINT (745158.923 4322394.234)  745158.922634  4.322394e+06  2020-01-29   
1  POINT (745158.057 4322394.207)  745158.056693  4.322394e+06  2020-01-29   
2  POINT (745156.359 4322393.043)  745156.359374  4.322393e+06  2020-01-29   
3  POINT (745155.424 4322395.236)  745155.424308  4.322395e+06  2020-01-29   
4  POINT (745155.355 4322397.456)  745155.355183  4.322397e+06  2020-01-29   

   value   type  instrument  
0   95.0  depth  magnaprobe  
1  102.0  depth  magnaprobe  
2  100.0  depth  magnaprobe  
3   98.0  depth  magnaprobe  
4   90.0  depth  magnaprobe  
4071 records returned!


# Download LayerData for Snow Pits within Bounding Box

We query all Layer Data, cast these values to a float, and then compute the average. Then the query is filtered to only the data type 'density'. The output rho_avg_all is the average density of all snow pits, we also filter the query again by site_id to extract the average density of pit 1S1. These density values are then printed to the screen for comparison.

In [6]:
# Import the connection function from the snowexsql library
from snowexsql.db import get_db

from sqlalchemy import inspect

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

In [7]:
# Import the class reflecting the points table in the db
from snowexsql.data import LayerData

# Import the function to investigate a table
from snowexsql.db import get_table_attributes

# Use the function to see what columns are available to use. 
db_columns = get_table_attributes(LayerData)

In [8]:
# Import the function to get connect to the db
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData
from snowexsql.conversions import query_to_geopandas, query_to_pandas
from snowexsql.db import get_table_attributes

# Import some other tools 
# from datetime import date
import datetime
from sqlalchemy import inspect
import shapely.geometry
from geoalchemy2.shape import from_shape
import geoalchemy2.functions as gfunc

# load the database
db_name = 'snow:hackweek@db.snowexdata.org/snowex'
# engine, session = get_db(db_name)

# print('SnowEx Database successfully loaded!')

In [ ]:
# Data Type 
data = LayerData

# Measurement type  
meas_type = "density" 

# Get a session
engine, session = get_db(db_name)

# The part inside the query function is what we want back, in this case all columns for the point data
qry = session.query(data.geom, data.easting, data.northing, data.date, data.value, 
                    data.type, data.instrument)

# Filter by bounding box 
qry = qry.filter(gfunc.ST_Within(PointData.geom, wkb_element))

# Filter by an data type 
qry = qry.filter(data.type == meas_type) # .in_(['magnaprobe', 'pit_ruler']))

# Slicing the dataset for specified dates 
date1 = datetime.date(2020,1,29)
date2 = datetime.date(2020,2,6)
qry = qry.filter(data.date.in_([date1, date2]))

# Execute the query and convert to geopandas in one handy function
df = query_to_geopandas(qry, engine) #directly pass to geopandas dataframe
print(df.head())

# how many did we retrieve?
print(f'{len(df.index)} records returned!')

session.close()

In [ ]:
# Request the average (avg) of Layer data casted as a float. We have to cast to a float in the layer table because all main values are stored as a string to
# ...accommodate the hand hardness.
qry = session.query(func.avg(LayerData.value.cast(Float)))
# Filter our query only to density
qry = qry.filter(LayerData.type=='density')
# Request the data
rho_avg_all = qry.all()
# Request the Average Density of Just 1S1
rho_avg_1s1 = qry.filter(LayerData.site_id == site_id).limit(1)
# This is a gotcha. The data in layer data only is stored as a string to accommodate the hand hardness values
print(f"Average density of all pits is {rho_avg_all[0][0]:0.0f} kg/m3")
print(f"Average density of pit 1S1 is {rho_avg_1s1[0][0]:0.0f} kg/m3")
# Cast Densities to float
rho_avg_all = float(rho_avg_all[0][0])
rho_avg_1s1 = float(rho_avg_1s1[0][0])

# IGNORE EVERYTHING PAST THIS POINT

### Create a Query

In [ ]:
# This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name)

# Lets grab a single row from the points table
qry = session.query(PointData)
qry = qry.filter(PointData.site_name == 'Grand Mesa')
count = qry.count() 
print(count) 

# Execute that query!
# result = qry.all()

session.close()

In [ ]:
# This is what you will use for all of hackweek to access the db
engine, session = get_db(db_name) # try to bracket this with session.close() as closely as possible as not to be kicked from the database 

# Its convenient to store a query like the following 
qry = session.query(PointData.instrument, PointData.date, PointData.site_name) 

# Then filter on it to just density profiles
qry = qry.filter(PointData.site_name == 'Grand Mesa') # pull data from location 
qry = qry.filter(PointData.instrument == 'magnaprobe') 
# qry = qry.filter(PointData.instrument == 'pit ruler') 
# qry = qry.filter(PointData.instrument == 'Mala 1600 MHz GPR') 
# qry = qry.filter(PointData.instrument == 'Mala 1600 MHz GPR') 
# qry = qry.filter(PointData.instrument == 'pulse EKKO Pro multi-polarization 1 GHz GPR') 


# protect ourselves from a lot of data
# qry = qry.limit(500) # only pull 5 points 

result = qry.all()
# print(result)

check = qry.filter(PointData.instrument == 'pit ruler')
print(check)

# Let see what instruments are available 
result = session.query(PointData.instrument).filter(PointData.type == 'depth').distinct().all()
print('\nInstruments Used to Measure Depth: ', result)

result = session.query(PointData.instrument).filter(PointData.type == 'two_way_travel').distinct().all()
print('\nInstruments Used to Measure Two Way Travel: ', result)

result = session.query(PointData.instrument).filter(PointData.type == 'swe').distinct().all()
print('\nInstruments Used to Measure SWE: ', result)

result = session.query(PointData.type).distinct().all()
print('\nInstrument Measurement Types: ', result)

result = session.query(PointData.instrument).distinct().all()
print('\nInstrument Types: ', result)

session.close()

### List Available Dates for Each Point Data Type 

In [ ]:
# df = pd.DataFrame(columns = len(session.query(PointData.instrument).filter(PointData.type == 'depth').distinct().all()))

results = session.query(PointData.date).filter(PointData.instrument == 'magnaprobe').distinct().all()
print('\nAvailable Dates for Magnaprobe = {}'.format(', '.join([str(r[0]) for r in results])))

results = session.query(PointData.date).filter(PointData.instrument == 'Mala 1600 MHz GPR').distinct().all()
print('\nAvailable Dates Mala 1600 MHz GPR = {}'.format(', '.join([str(r[0]) for r in results])))

results = session.query(PointData.date).filter(PointData.instrument == 'Mala 800 MHz GPR').distinct().all()
print('\nAvailable Dates Mala 800 MHz GPR = {}'.format(', '.join([str(r[0]) for r in results])))

results = session.query(PointData.date).filter(PointData.instrument == 'pulse EKKO Pro multi-polarization 1 GHz GPR').distinct().all()
print('\nAvailable Dates pulse EKKO Pro multi-polarization 1 GHz GPR = {}'.format(', '.join([str(r[0]) for r in results])))

results = session.query(PointData.date).filter(PointData.instrument == 'pit ruler').distinct().all()
print('\nAvailable Dates pit ruler = {}'.format(', '.join([str(r[0]) for r in results])))


In [ ]:
results = session.query(SiteData.date).filter(SiteData.instrument == 'magnaprobe').distinct().all()
print('\nAvailable Dates for Magnaprobe = {}'.format(', '.join([str(r[0]) for r in results])))
